# How to run optimization on PyRICE model

This notebook provides an example of how to run the Optimization on PyRICE model and shows the main parameters and their domains.

---

In [2]:
#Setting directory

import os
try:
    os.chdir(os.path.join(os.getcwd(), '/Users/palokbiswas/Desktop/RICE/PyRICE_Max_New/PyRICE_2022')) # '.' if the path is to current folder
    print(os.getcwd())
except:
    pass

/Users/palokbiswas/Desktop/RICE/PyRICE_Max_New/PyRICE_2022


## 1. Imports

This module contains functions that support the run of an optimization.


In [3]:
# Imports

from model.pyrice import PyRICE
from model.enumerations import *
from dmdu.general.xlm_constants_epsilons import (
    get_outcomes_and_epsilons,
    get_xlc,
)

from ema_workbench import ScalarOutcome, RealParameter, IntegerParameter, Constant
import os

# EMA
from ema_workbench.em_framework.optimization import EpsilonProgress, ArchiveLogger
from ema_workbench import Model, MultiprocessingEvaluator, ema_logging, RealParameter

ema_logging.log_to_stderr(ema_logging.INFO)

from platypus import Real

import itertools




In [4]:
#Testing EMA RealParameter & Platypus Real

types = []

types.append(Real(0,1))

print(types[0])
types_2 = []
types_2.append(RealParameter("c1", 0, 1))
print(types_2[0])


n_inputs = 2  # (time, storage of Conowingo)
n_outputs = 4
n_rbfs = 4
            # rbf = rbf_functions.RBF(n_rbfs, n_inputs, n_outputs, rbf_function=entry)

types = []
c_i = []
r_i = []
w_i = []
count = itertools.count()

for i in range(n_rbfs): #4
    for j in range(n_inputs): #2
        types.append(Real(-1, 1))  # center
        c_i.append(next(count))
        types.append(Real(0, 1))  # radius
        r_i.append(next(count))

for _ in range(n_rbfs): #4
    for _ in range(n_outputs): #4
        types.append(Real(0, 1))  # weight
        w_i.append(next(count))  # weight

print(c_i)

print(r_i)

print(w_i)

print(len(types))

"""
Real(0.000000, 1.000000)
c1
[0, 2, 4, 6, 8, 10, 12, 14]
[1, 3, 5, 7, 9, 11, 13, 15]
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
32


#f-strings
lake_model.levers = [RealParameter(f"l{i}", 0, 0.1) for i in range(lake_model.time_horizon)]
"""

Real(0.000000, 1.000000)
c1
[0, 2, 4, 6, 8, 10, 12, 14]
[1, 3, 5, 7, 9, 11, 13, 15]
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
32


'\nReal(0.000000, 1.000000)\nc1\n[0, 2, 4, 6, 8, 10, 12, 14]\n[1, 3, 5, 7, 9, 11, 13, 15]\n[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]\n32\n\n\n#f-strings\nlake_model.levers = [RealParameter(f"l{i}", 0, 0.1) for i in range(lake_model.time_horizon)]\n'

In [31]:
#print(f"l{i}" for i in range (10))

print([f"w{i}" for i in range (37)]) #iterates through the dictionary OBJECT and sets it to the value 0

"""
w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18, w19, w20, w21, w22, w23, w24, w25, w26, w27, w28, w29, w30, w31, w32, w33, w34, w35, w36
"""

['w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'w13', 'w14', 'w15', 'w16', 'w17', 'w18', 'w19', 'w20', 'w21', 'w22', 'w23', 'w24', 'w25', 'w26', 'w27', 'w28', 'w29', 'w30', 'w31', 'w32', 'w33', 'w34', 'w35', 'w36']


In [24]:
import numpy as np
c_i = np.asarray([[0.5], [0.5], [0.5]])



w_i = np.asarray([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  
                  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  
                  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


inp =  np.asarray([1])

print(inp.shape)


(1,)


In [5]:
def define_path_name(problem_formulation, nfe, seed_index, ref_index, directory=None, d_type="results", searchover=None):
    """
    Define path and file name such that it can be used to save results_formatted and/or covergence outcomes.
    @param problem_formulation: ProblemFormulation
    @param nfe: integer
    @param seed_index: int
    @param ref_index: int: number of current reference scenario/policy
    @param directory: String: where to save results and covergence outcomes
    @param d_type: string: {'results_formatted', 'convergence'}
    @param searchover: String
    @return:
        path: string (path + file name that is used for saving_results)
    """

    if d_type == 'results' or d_type == 'epsilon_progress':
        file_name = f'{d_type}.csv'
    elif d_type == 'hypervolume':
        file_name = ''
    else:
        raise ValueError(
            'You passed an unvalid d_type in order to save your resulting outcomes.'
        )

    if directory is None:
        directory = get_directory(d_type, searchover, seed_index, problem_formulation, nfe, ref_index)

    path = os.path.join(directory, file_name)

    return path

In [6]:
def get_directory(d_type, searchover, seed_index, problem_formulation, nfe, ref_index):
    """
    Create a directory if necessary.
    @return:
        path: string
    """
    reference_name = 'reference_scenario' if searchover == 'levers' else 'reference_policy'

    directory = os.path.abspath(os.getcwd())
    data_folder = 'data'
    problem_folder = f'{problem_formulation.name}_{nfe}'
    seed_folder = f'seed_{seed_index}'
    scenario_folder = f'{reference_name}_{ref_index}'

    if d_type == 'hypervolume':
        sub_folder = d_type
        path = os.path.join(directory, data_folder, problem_folder, seed_folder, scenario_folder, sub_folder)
    else:
        path = os.path.join(directory, data_folder, problem_folder, seed_folder, scenario_folder)

    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except OSError:
            print("Creation of the directory failed")
            raise

    return path

In [7]:
def run_optimization(
    damage_function=DamageFunction.NORDHAUS,
    problem_formulation=ProblemFormulation.UTILITARIAN_AGGREGATED,
    nfe=5000,
    searchover='levers',
    seed_index=None,
    reference=None,
    saving_results=False,
    with_convergence=False,
):
    """
    This function runs an optimization with the PyRICE model.
    @param damage_function: DamageFunction
    @param problem_formulation: ProblemFormulation
    @param nfe: integer
    @param searchover: String: {'levers', 'uncertainties'}
    @param seed_index: int
    @param reference: tuple with (index, Scenario/Policy object)
    @param saving_results: Boolean: whether to save results_formatted or not
    @param with_convergence: Boolean: whether to save convergence outcomes or not
    """
    welfare_function, aggregation, _ = problem_formulation.value

    # Instantiate the model
    model_specification = ModelSpec.STANDARD

    model = PyRICE(
        model_specification=model_specification,
        damage_function=damage_function,
        welfare_function=welfare_function,
    )

    model = Model("RICE", function=model)
    
    levers = [
        RealParameter("sr", 0.1, 0.5),
        #RealParameter("miu", 2065, 2300),
        RealParameter("irstp_consumption", 0.001, 0.015),
        RealParameter("irstp_damage", 0.001, 0.015),

        RealParameter("c1", -1.0, 1.0),
        RealParameter("c2", -1.0, 1.0),
        RealParameter("c3", -1.0, 1.0),

        RealParameter("r1", 0.0, 1.0),
        RealParameter("r2", 0.0, 1.0),
        RealParameter("r3", 0.0, 1.0),


        RealParameter("w0", 0.0, 1.0),
        RealParameter("w1", 0.0, 1.0),
        RealParameter("w2", 0.0, 1.0),
        RealParameter("w3", 0.0, 1.0),
        RealParameter("w4", 0.0, 1.0),
        RealParameter("w5", 0.0, 1.0),
        RealParameter("w6", 0.0, 1.0),
        RealParameter("w7", 0.0, 1.0),
        RealParameter("w8", 0.0, 1.0),
        RealParameter("w9", 0.0, 1.0),
        RealParameter("w10", 0.0, 1.0),
        RealParameter("w11", 0.0, 1.0),

        RealParameter("w12", 0.0, 1.0),
        RealParameter("w13", 0.0, 1.0),
        RealParameter("w14", 0.0, 1.0),
        RealParameter("w15", 0.0, 1.0),
        RealParameter("w16", 0.0, 1.0), 
        RealParameter("w17", 0.0, 1.0),  
        RealParameter("w18", 0.0, 1.0),
        RealParameter("w19", 0.0, 1.0),
        RealParameter("w20", 0.0, 1.0),
        RealParameter("w21", 0.0, 1.0),
        RealParameter("w22", 0.0, 1.0), 
        RealParameter("w23", 0.0, 1.0),  

        RealParameter("w24", 0.0, 1.0),
        RealParameter("w25", 0.0, 1.0),
        RealParameter("w26", 0.0, 1.0),
        RealParameter("w27", 0.0, 1.0),
        RealParameter("w28", 0.0, 1.0), 
        RealParameter("w29", 0.0, 1.0),  
        RealParameter("w30", 0.0, 1.0),
        RealParameter("w31", 0.0, 1.0),
        RealParameter("w32", 0.0, 1.0),
        RealParameter("w33", 0.0, 1.0),
        RealParameter("w34", 0.0, 1.0), 
        RealParameter("w35", 0.0, 1.0),

        #[RealParameter(f"w{i}", 0, 1.0) for i in range(31)],


    ]

    uncertainties = [
        IntegerParameter("t2xco2_index", 0, 999),
        IntegerParameter("t2xco2_dist", 0, 2),
        RealParameter("fosslim", 4000, 13649),
        IntegerParameter("scenario_pop_gdp", 0, 5),
        IntegerParameter("scenario_sigma", 0, 2),
        IntegerParameter("scenario_cback", 0, 1),
        IntegerParameter("scenario_elasticity_of_damages", 0, 2),
        IntegerParameter("scenario_limmiu", 0, 1),
        RealParameter("emdd", 0.001, 0.6)
    ]

    constants = [Constant("precision", 10)]

    model.uncertainties = uncertainties
    model.levers = levers
    model.constants = constants
    
    model.outcomes, epsilons = get_outcomes_and_epsilons(problem_formulation=problem_formulation, searchover=searchover)

    # look at reference
    if reference is None:
        ref_index = 0
    else:
        ref_index = reference[0]
        reference = reference[1]

    # Run optimization
    if with_convergence:

        directory = define_path_name(
            problem_formulation=problem_formulation,
            nfe=nfe,
            seed_index=seed_index,
            ref_index=ref_index,
            d_type="hypervolume",
            searchover=searchover
        )
        convergence_metrics = [
            EpsilonProgress(),
            ArchiveLogger(
                directory,
                [l.name for l in model.levers],
                [o.name for o in model.outcomes if o.kind != o.INFO],
            ),
        ]

        with MultiprocessingEvaluator(model) as evaluator:
            results, convergence = evaluator.optimize(
                nfe=nfe,
                searchover=searchover,
                epsilons=epsilons,
                convergence=convergence_metrics,
                reference=reference
            )

            if saving_results:
                # Save results_formatted
                path = define_path_name(
                    problem_formulation=problem_formulation,
                    nfe=nfe,
                    seed_index=seed_index,
                    ref_index=ref_index,
                    d_type="results",
                    searchover=searchover,
                )
                results.to_csv(path)

                # Save convergence
                path = define_path_name(
                    problem_formulation=problem_formulation,
                    nfe=nfe,
                    seed_index=seed_index,
                    ref_index=ref_index,
                    d_type="epsilon_progress",
                    searchover=searchover,
                )
                convergence.to_csv(path)

    else:

        with MultiprocessingEvaluator(model, n_processes=50) as evaluator:
            results = evaluator.optimize(
                nfe=nfe,
                searchover=searchover,
                epsilons=epsilons,
                reference=reference
            )

            if saving_results:
                path = define_path_name(
                    problem_formulation=problem_formulation,
                    nfe=nfe,
                    seed_index=seed_index,
                    ref_index=ref_index,
                    d_type="results",
                    searchover=searchover,
                )
                results.to_csv(path)

In [8]:
#changed from nfe 200000 to 5000 & searchover uncertainties to levers
if __name__ == "__main__":

    run_optimization(
        damage_function=DamageFunction.NORDHAUS,
        problem_formulation=ProblemFormulation.UTILITARIAN_AGGREGATED,
        nfe=1,
        searchover="levers",
        saving_results=True,
        with_convergence=False,
    )

[MainProcess/INFO] pool started with 50 workers
  0%|                                                    | 0/1 [00:00<?, ?it/s]

checking temp_atm max
40.0
printing t: 
1
checking temp_atm max
40.0
printing t: 
1
checking temp_atm max
40.0
printing t: 
1
checking temp_atm max
40.0
printing t: 
1
checking temp_atm max
40.0
printing t: 
1
checking temp_atm max
40.0
printing t: 
1
checking temp_atm max
40.0
printing t: 
1
checking temp_atm max
40.0
printing t: 
1


'PyRICE' object has no attribute 'c1'
Traceback (most recent call last):
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 403, in run_experiment
    model_output = self.function(**experiment)
  File "/Users/palokbiswas/Desktop/RICE/PyRICE_Max_New/PyRICE_2022/model/pyrice.py",

checking temp_atm max
40.0
printing t: 
1
checking temp_atm max
40.0
printing t: 
1


'PyRICE' object has no attribute 'c1'
Traceback (most recent call last):
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 403, in run_experiment
    model_output = self.function(**experiment)
  File "/Users/palokbiswas/Desktop/RICE/PyRICE_Max_New/PyRICE_2022/model/pyrice.py",

...

'PyRICE' object has no attribute 'c1'
Traceback (most recent call last):
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 403, in run_experiment
    model_output = self.function(**experiment)
  File "/Users/palokbiswas/Desktop/RICE/PyRICE_Max_New/PyRICE_2022/model/pyrice.py",

checking temp_atm max
40.0
printing t: 
1


'PyRICE' object has no attribute 'c1'
Traceback (most recent call last):
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 403, in run_experiment
    model_output = self.function(**experiment)
  File "/Users/palokbiswas/Desktop/RICE/PyRICE_Max_New/PyRICE_2022/model/pyrice.py",

checking temp_atm max
40.0
printing t: 
1


'PyRICE' object has no attribute 'c1'
Traceback (most recent call last):
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/palokbiswas/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 403, in run_experiment
    model_output = self.function(**experiment)
  File "/Users/palokbiswas/Desktop/RICE/PyRICE_Max_New/PyRICE_2022/model/pyrice.py",